In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/content/chipotle.tsv", sep="\t")

In [3]:
df.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


# 1) For Missing Values

In [4]:
df.isnull().sum()

order_id                 0
quantity                 0
item_name                0
choice_description    1246
item_price               0
dtype: int64

# 2) Data Types

In [5]:
df.dtypes

order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

# 3) Quantity and Itom Prices

In [6]:
df["quantity"].unique()

array([ 1,  2,  3,  4,  5, 15,  7,  8, 10])

In [7]:
df["item_price"].unique()

array(['$2.39 ', '$3.39 ', '$16.98 ', '$10.98 ', '$1.69 ', '$11.75 ',
       '$9.25 ', '$4.45 ', '$8.75 ', '$11.25 ', '$8.49 ', '$2.18 ',
       '$8.99 ', '$1.09 ', '$2.95 ', '$2.15 ', '$3.99 ', '$22.50 ',
       '$11.48 ', '$17.98 ', '$17.50 ', '$4.30 ', '$5.90 ', '$1.25 ',
       '$23.78 ', '$6.49 ', '$11.08 ', '$1.50 ', '$22.16 ', '$32.94 ',
       '$22.20 ', '$10.58 ', '$2.50 ', '$23.50 ', '$7.40 ', '$18.50 ',
       '$3.00 ', '$6.78 ', '$11.89 ', '$9.39 ', '$4.00 ', '$3.75 ',
       '$8.69 ', '$2.29 ', '$8.90 ', '$3.27 ', '$3.89 ', '$8.19 ',
       '$35.00 ', '$27.75 ', '$11.80 ', '$6.00 ', '$26.25 ', '$21.96 ',
       '$4.36 ', '$7.50 ', '$4.78 ', '$13.35 ', '$6.45 ', '$5.07 ',
       '$22.96 ', '$7.17 ', '$7.98 ', '$4.50 ', '$26.07 ', '$12.98 ',
       '$35.25 ', '$44.25 ', '$10.50 ', '$33.75 ', '$16.38 ', '$13.52 ',
       '$5.00 ', '$15.00 ', '$8.50 ', '$17.80 ', '$1.99 ', '$11.49 '],
      dtype=object)

In [8]:
df["item_price"] = df["item_price"].str.replace("$","")
df["item_price"] = df["item_price"].astype(float).round(decimals = 2)
for i in df.index:
  if df.loc[i,"quantity"] >1:
    for j in df.index:
      if i == j:
        continue
      elif df.loc[i,"item_name"] == df.loc[j,"item_name"] and df.loc[i,"choice_description"] == df.loc[j,"choice_description"]:
        if df.loc[i,"item_price"] == df.loc[i,"quantity"]*df.loc[j,"item_price"]:
          continue
        else:
          df.loc[i,"item_price"] = df.loc[i,"quantity"]*df.loc[j,"item_price"]
          break

<ipython-input-8-05feeb2ea0b6>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["item_price"] = df["item_price"].str.replace("$","")


In [9]:
df.head(15)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39
1,1,1,Izze,[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",10.98
6,3,1,Side of Chips,NaN,1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",9.25


# Duplicate Entries

In [10]:
df.duplicated().sum()

59

In [11]:
df = df.drop_duplicates(subset = ["order_id","quantity","item_name","choice_description"])

In [12]:
df.duplicated().sum()

0

# 4) Order ID Integrity

In [13]:
df.reset_index(inplace = True)

In [14]:
rows_to_drop = []

for x in range(len(df) - 1):
    if df.loc[x, "order_id"] <= df.loc[x + 1, "order_id"]:
        continue
    else:
        rows_to_drop.append(x)


df = df.drop(index=rows_to_drop).reset_index(drop=True)

In [15]:
df.head(10)
df.columns

Index(['index', 'order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')

# Choice Description

In [19]:
choice_discription_list = []
choice_discription_list = df["choice_description"].unique()
print(choice_discription_list)

[nan '[Clementine]' '[Apple]' ...
 '[Roasted Chili Corn Salsa, [Pinto Beans, Sour Cream, Cheese, Lettuce, Guacamole]]'
 '[Tomatillo Green Chili Salsa, [Rice, Black Beans]]'
 '[Tomatillo Green Chili Salsa, [Rice, Fajita Vegetables, Black Beans, Guacamole]]']


In [20]:
df["choice_description"] = df["choice_description"].fillna("Default")

In [21]:
df.head(5)

,index,order_id,quantity,item_name,choice_description,item_price
0,0,1,1,Chips and Fresh Tomato Salsa,Default,2.39
1,1,1,1,Izze,[Clementine],3.39
2,2,1,1,Nantucket Nectar,[Apple],3.39
3,3,1,1,Chips and Tomatillo Green Chili Salsa,Default,2.39
4,4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98


# 5) Item Name Standerdization

In [16]:
item_name_list = []
item_name_list = df["item_name"].unique()
print(item_name_list)

['Chips and Fresh Tomato Salsa' 'Izze' 'Nantucket Nectar'
 'Chips and Tomatillo-Green Chili Salsa' 'Chicken Bowl' 'Side of Chips'
 'Steak Burrito' 'Steak Soft Tacos' 'Chips and Guacamole'
 'Chicken Crispy Tacos' 'Chicken Soft Tacos' 'Chicken Burrito'
 'Canned Soda' 'Barbacoa Burrito' 'Carnitas Burrito' 'Carnitas Bowl'
 'Bottled Water' 'Chips and Tomatillo Green Chili Salsa' 'Barbacoa Bowl'
 'Chips' 'Chicken Salad Bowl' 'Steak Bowl' 'Barbacoa Soft Tacos'
 'Veggie Burrito' 'Veggie Bowl' 'Steak Crispy Tacos'
 'Chips and Tomatillo Red Chili Salsa' 'Barbacoa Crispy Tacos'
 'Veggie Salad Bowl' 'Chips and Roasted Chili-Corn Salsa'
 'Chips and Roasted Chili Corn Salsa' 'Carnitas Soft Tacos'
 'Chicken Salad' 'Canned Soft Drink' 'Steak Salad Bowl'
 '6 Pack Soft Drink' 'Chips and Tomatillo-Red Chili Salsa' 'Bowl'
 'Burrito' 'Crispy Tacos' 'Carnitas Crispy Tacos' 'Steak Salad'
 'Chips and Mild Fresh Tomato Salsa' 'Veggie Soft Tacos'
 'Carnitas Salad Bowl' 'Barbacoa Salad Bowl' 'Salad' 'Veggie Cris

In [17]:
df["item_name"] = df["item_name"].str.replace("-"," ")

In [18]:
df.head(5)

,index,order_id,quantity,item_name,choice_description,item_price
0,0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39
1,1,1,1,Izze,[Clementine],3.39
2,2,1,1,Nantucket Nectar,[Apple],3.39
3,3,1,1,Chips and Tomatillo Green Chili Salsa,NaN,2.39
4,4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98


# 6) Handling Categorial Data

In [22]:
# Once the data is cleaned, exporting the cleaned data so that we can check its data integrity
df.to_csv('Dataset-2-cleaned_data.csv')

In [26]:
# Loading the cleaned data into pandas dataframe
df_uncleaned = pd.read_csv("/content/chipotle.tsv" , sep = "\t")
df_cleaned = pd.read_csv("/content/Dataset-2-cleaned_data.csv")

In [27]:
print(df_cleaned.head())
print(df_uncleaned.head())

   Unnamed: 0  index  order_id  quantity  \
0           0      0         1         1   
1           1      1         1         1   
2           2      2         1         1   
3           3      3         1         1   
4           4      4         2         2   

                               item_name  \
0           Chips and Fresh Tomato Salsa   
1                                   Izze   
2                       Nantucket Nectar   
3  Chips and Tomatillo Green Chili Salsa   
4                           Chicken Bowl   

                                  choice_description  item_price  
0                                            Default        2.39  
1                                       [Clementine]        3.39  
2                                            [Apple]        3.39  
3                                            Default        2.39  
4  [Tomatillo-Red Chili Salsa (Hot), [Black Beans...       16.98  
   order_id  quantity                              item_name  \
0   

In [28]:
print(df_uncleaned.columns)
print(df_cleaned.columns)

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')
Index(['Unnamed: 0', 'index', 'order_id', 'quantity', 'item_name',
       'choice_description', 'item_price'],
      dtype='object')


In [29]:
df_cleaned = df_cleaned.drop ('index', axis=1)
df_cleaned = df_cleaned.drop ('Unnamed: 0', axis=1)

In [30]:
# Summarize the statistics of the DataFrames
print("Summary of uncleaned data:")
print(df_uncleaned.describe())
print()

print("Summary of cleaned data:")
print(df_cleaned.describe())
print()

Summary of uncleaned data:
          order_id     quantity
count  4622.000000  4622.000000
mean    927.254868     1.075725
std     528.890796     0.410186
min       1.000000     1.000000
25%     477.250000     1.000000
50%     926.000000     1.000000
75%    1393.000000     1.000000
max    1834.000000    15.000000

Summary of cleaned data:
          order_id     quantity   item_price
count  4563.000000  4563.000000  4563.000000
mean    927.045803     1.076704     8.041034
std     528.936100     0.412739     7.720611
min       1.000000     1.000000     1.090000
25%     476.500000     1.000000     3.990000
50%     926.000000     1.000000     8.750000
75%    1393.000000     1.000000    10.780000
max    1834.000000    15.000000   211.500000



In [31]:

print("Missing values in uncleaned data:")
print(df_uncleaned.isnull().sum())
print()

print("Missing values in cleaned data:")
print(df_cleaned.isnull().sum())
print()

print("Duplicated values in uncleaned data:")
print(df_uncleaned.duplicated().sum())
print()

print("Duplicated values in cleaned data:")
print(df_cleaned.duplicated().sum())
print()

Missing values in uncleaned data:
order_id                 0
quantity                 0
item_name                0
choice_description    1246
item_price               0
dtype: int64

Missing values in cleaned data:
order_id              0
quantity              0
item_name             0
choice_description    0
item_price            0
dtype: int64

Duplicated values in uncleaned data:
59

Duplicated values in cleaned data:
0



# 7) Converting to csv

In [36]:
df_csv = df.to_csv("/content/chipotle.tsv",sep = ',')